## Using BigQuery ML to Predict Taxi Fares

### April 7, 2019, Team 4 

<b>Kush Shah, Pallav Jhaveri, Darshan Shah, Anusha Balaji </b>

In [13]:
!pip install --upgrade google-cloud-bigquery

Requirement already up-to-date: google-cloud-bigquery in /usr/local/envs/py2env/lib/python2.7/site-packages (1.11.2)


In [14]:
from google.cloud import bigquery
client = bigquery.Client()

In [15]:
dataset = bigquery.Dataset(client.dataset('bqml_example'))
dataset.location = 'US'
#client.create_dataset(dataset)

In [16]:
client.get_dataset("bqml_example")

Dataset(DatasetReference(u'bq-tutorial-april-2019', u'bqml_example'))

In [17]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [22]:
%%bigquery
CREATE OR REPLACE MODEL bqml_example.taxi_model OPTIONS(
  model_type='linear_reg', max_iteration=10) AS
SELECT
  miles_range_max as miles_range_max, 
  CAST(average_fare AS FLOAT64),
  CAST(average_fare AS FLOAT64) as label
FROM
  bqml_example.Taxi_Miles_Fares
WHERE
  # remove the miles_range test point to predict/test our model
  miles_range != '39m to 54m'

""


In [19]:
%%bigquery
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `bqml_example.taxi_model`)

,training_run,iteration,loss,eval_loss,duration_ms,learning_rate
0,0,9,5.803622e-07,None,3303,0.4
1,0,8,1.450905e-05,None,2829,0.4
2,0,7,3.627264e-04,None,2516,0.4
3,0,6,9.068159e-03,None,4074,0.4
4,0,5,2.267040e-01,None,2813,0.4
5,0,4,5.667600e+00,None,2952,0.4
6,0,3,1.416900e+02,None,2636,0.4
7,0,2,3.542250e+03,None,2738,0.4
8,0,1,8.855624e+04,None,2486,0.4
9,0,0,2.213906e+06,None,3204,0.2


In [20]:
%%bigquery
WITH eval_table AS (
SELECT
  *,
  CAST(average_fare AS float64) AS label
FROM
  `bqml_example.Taxi_Miles_Fares` )
SELECT
  *
FROM
  ML.EVALUATE(MODEL `bqml_example.taxi_model`,
    TABLE eval_table)

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,1172.734095,2.332156e+06,0.266355,984.279653,0.150499,0.651462


In [21]:
%%bigquery
  WITH trip_to_predict AS (
  SELECT
    *
  FROM
    `bqml_example.Taxi_Miles_Fares`
  WHERE
    miles_range='39m to 54m' )
SELECT
  truth.miles_range AS miles_range,
  average_fare,
  predicted_average_fare
FROM (
  SELECT
    miles_range,
    predicted_label AS predicted_average_fare
  FROM
    ML.PREDICT(MODEL `bqml_example.taxi_model`,
      table trip_to_predict) ) AS predict
JOIN (
  SELECT
    miles_range,
    average_fare AS average_fare
  FROM
    trip_to_predict) AS truth
ON
  predict.miles_range = truth.miles_range

,miles_range,average_fare,predicted_average_fare
0,39m to 54m,5385.76,1211.493627
